In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.plot_utils import dafx_from_name
from sklearn.feature_selection import mutual_info_regression

In [2]:
sns.set(style="white")

In [3]:
DATA_DIR = "/home/kieran/Level5ProjectAudioVAE/src/evaluation/data/param_extraction"
FIG_DIR = "/home/kieran/Level5ProjectAudioVAE/src/evaluation/figures/param_entropy_multi"
OUT_DATA_DIR = "/home/kieran/Level5ProjectAudioVAE/src/evaluation/data/param_entropy_multi"

DAFX = ["overdrive", "combo", "delay", "ambience", "dynamics", "ringmod"]

DAFX_TO_MDA_NAME = {
    "delay": "mda Delay",
    "combo": "mda Combo",
    "overdrive": "mda Overdrive",
    "ambience": "mda Ambience",
    "dynamics": "mda Dynamics",
    "ringmod": "mda RingMod",
}

In [29]:
dataframes = {}

param_sizes = {}

for fx in DAFX:
    emb = np.load(f"{DATA_DIR}/{fx}_projection.npy")
    params = np.load(f"{DATA_DIR}/{fx}_settings.npy")

    idx_param_map = dafx_from_name(DAFX_TO_MDA_NAME[fx]).idx_to_param_map

    param_sizes[fx] = len(idx_param_map)

    mi_matrix = np.array([mutual_info_regression(emb, params[:, i]) for i in range(params.shape[1])])
    df = pd.DataFrame(mi_matrix, index=[idx_param_map[i] for i in range(params.shape[1])])

    dataframes[fx] = df

    df.to_csv(f"{OUT_DATA_DIR}/{fx}.csv")

In [30]:
sorted_fx = sorted(param_sizes, key=param_sizes.get)

In [97]:
subframes = []
vmax = 0

for fx in sorted_fx:
    df = dataframes[fx]
    sub_df = df.max(axis='columns').reset_index()

    vm = sub_df[0].max()

    vmax = max(vm, vmax)

    # mean_data = ['mean', sub_df[0].mean()]
    # sub_df = pd.concat([sub_df, pd.DataFrame([mean_data], columns=sub_df.columns)], ignore_index=True)

    fx_name = DAFX_TO_MDA_NAME[fx].split()[-1]

    y = [(fx_name, 'Param'), (fx_name, 'MMI')]
    cols = pd.MultiIndex.from_tuples(y)
    sub_df.columns = cols

    subframes.append(sub_df)

In [98]:
vmax

3.3155506127469057

In [99]:
full_df = pd.concat(subframes, axis=1)

In [100]:
full_df.to_csv(f"{OUT_DATA_DIR}/full.csv")

In [101]:
slice = [x for x in full_df.columns if x[1] == 'MMI']

In [102]:
slice

[('Overdrive', 'MMI'),
 ('RingMod', 'MMI'),
 ('Ambience', 'MMI'),
 ('Combo', 'MMI'),
 ('Delay', 'MMI'),
 ('Dynamics', 'MMI')]

In [103]:
blue_cm = sns.light_palette("blue", as_cmap=True)

display(full_df.style.background_gradient(cmap=blue_cm, subset=slice, vmin=0, vmax=vmax))



In [94]:
full_df

Overdrive             RingMod             Ambience                Combo  \
        Param       MMI     Param       MMI      Param       MMI      Param   
0       drive  0.580494   freq_hz  3.315551     size_m  0.306228  drive_s_h   
1      muffle  1.378618   fine_hz  0.029495    hf_damp  0.022788       bias   
2   output_db  0.001248  feedback  0.138130        mix  0.474104  output_db   
3        mean  0.653453      mean  1.161059  output_db  0.006581   hpf_freq   
4         NaN       NaN       NaN       NaN       mean  0.202425   hpf_reso   
5         NaN       NaN       NaN       NaN        NaN       NaN       mean   
6         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
7         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
8         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
9         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
10        NaN       NaN       NaN       NaN        NaN       NaN        NaN   

                      Delay               Dynamics            
         MMI          Param       MMI        Param       MMI  
0   0.891225     l_delay_ms  0.789228    thresh_db  0.021160  
1   0.010484        r_delay  0.222316        ratio  0.009682  
2   0.005743       feedback  0.030766    output_db  0.070965  
3   1.582003  fb_tone_lo_hi  0.047116     attack_s  0.000000  
4   0.225313         fb_mix  0.318375   release_ms  0.178540  
5   0.542954           mean  0.281560   limiter_db  0.018847  
6        NaN            NaN       NaN  gate_thr_db  0.249928  
7        NaN            NaN       NaN   gate_att_s  0.000000  
8        NaN            NaN       NaN  gate_rel_ms  0.074006  
9        NaN            NaN       NaN          mix  0.064735  
10       NaN            NaN       NaN         mean  0.068786

In [73]:
full_df

Overdrive             RingMod             Ambience                Combo  \
        Param       MMI     Param       MMI      Param       MMI      Param   
0       drive  0.580494   freq_hz  3.315551     size_m  0.306228  drive_s_h   
1      muffle  1.378618   fine_hz  0.029495    hf_damp  0.022788       bias   
2   output_db  0.001248  feedback  0.138130        mix  0.474104  output_db   
3        mean  0.653453      mean  1.161059  output_db  0.006581   hpf_freq   
4         NaN       NaN       NaN       NaN       mean  0.202425   hpf_reso   
5         NaN       NaN       NaN       NaN        NaN       NaN       mean   
6         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
7         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
8         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
9         NaN       NaN       NaN       NaN        NaN       NaN        NaN   
10        NaN       NaN       NaN       NaN        NaN       NaN        NaN   

                      Delay               Dynamics            
         MMI          Param       MMI        Param       MMI  
0   0.891225     l_delay_ms  0.789228    thresh_db  0.021160  
1   0.010484        r_delay  0.222316        ratio  0.009682  
2   0.005743       feedback  0.030766    output_db  0.070965  
3   1.582003  fb_tone_lo_hi  0.047116     attack_s  0.000000  
4   0.225313         fb_mix  0.318375   release_ms  0.178540  
5   0.542954           mean  0.281560   limiter_db  0.018847  
6        NaN            NaN       NaN  gate_thr_db  0.249928  
7        NaN            NaN       NaN   gate_att_s  0.000000  
8        NaN            NaN       NaN  gate_rel_ms  0.074006  
9        NaN            NaN       NaN          mix  0.064735  
10       NaN            NaN       NaN         mean  0.068786